# Pipelines: Metaflow model training

##  Install dependencies

In [ ]:
#!pip install -r requirements.txt

## Set username

In [ ]:
# Set username for workflows
import os
os.environ["USERNAME"] = "daniel"

In [ ]:
%%writefile metaflow_trainingflow.py
from metaflow import FlowSpec, Parameter, step

class TrainingFlow(FlowSpec):
    # Define the parameters for the flow
    max_depth = Parameter('max_depth', default=2, help='Max depth of the random forest classifier')
    n_estimators = Parameter('n_estimators', default=100, help='Number of estimators for the random forest classifier')
    random_state = Parameter('random_state', default=0, help='Random state for the random forest classifier')
    
    @step
    def start(self):
        # Start the flow
        self.next(self.ingest_data)
        
    @step
    def ingest_data(self):
        from sklearn.datasets import load_iris
    
        # Load the iris dataset
        iris = load_iris()
        
        #pylint: disable=no-member
        self.X = iris.data
        self.y = iris.target
        #pylint: enable=no-member
        
        self.next(self.split_data)

    @step
    def split_data(self):
        #Split the data into train and test
        # TODO: WRITE YOUR CODE HERE

    @step
    def train(self):
        # Train the model
        # TODO: WRITE YOUR CODE HERE

    @step
    def show_metrics(self):
        # Print some metrics
        # TODO: WRITE YOUR CODE HERE
        
    @step
    def register_model(self):
        # Save the model in a pickle file in local storage
        # TODO: WRITE YOUR CODE HERE
        
    @step
    def end(self):
        pass
    
if __name__ == '__main__':
    TrainingFlow()

In [ ]:
!python metaflow_trainingflow.py run --max_depth 2 --n_estimators 100 --random_state 0